<div dir="rtl">
    <font color=blue size=6>
تحلیل دیتاست نوع تومور سینه با تکنیک نایو بیز 
    </font>
</div>

<div dir="rtl">
    <font size=4>
    اینجا
    </font>
</div>

In [1]:
import pandas as pd

In [2]:
from sklearn.preprocessing import OrdinalEncoder

In [3]:
from sklearn.metrics import f1_score

<div dir="rtl">
    <font size=4>
    ورودی ها را تمامی به فرمت دسته ای تبدیل کرده و هدف هم به صورت دودویی جدا شده است
    </font>
</div>

In [4]:
db = pd.read_csv('./breast-cancer.data', header=None,
                names=['recurrence-events', 'age', 'menopause', 'tumor-size', 'inv-nodes',
                       'node-caps', 'deg-malig', 'breast', 'breast-quad', 'irradiat'],
                true_values=['recurrence-events'],
                false_values=['no-recurrence-events'])
db

,recurrence-events,age,menopause,tumor-size,inv-nodes,node-caps,deg-malig,breast,breast-quad,irradiat
0,False,30-39,premeno,30-34,0-2,no,3,left,left_low,no
1,False,40-49,premeno,20-24,0-2,no,2,right,right_up,no
2,False,40-49,premeno,20-24,0-2,no,2,left,left_low,no
3,False,60-69,ge40,15-19,0-2,no,2,right,left_up,no
4,False,40-49,premeno,0-4,0-2,no,2,right,right_low,no
...,...,...,...,...,...,...,...,...,...,...
281,True,30-39,premeno,30-34,0-2,no,2,left,left_up,no
282,True,30-39,premeno,20-24,0-2,no,3,left,left_up,yes
283,True,60-69,ge40,20-24,0-2,no,1,right,left_up,no
284,True,40-49,ge40,30-34,3-5,no,3,left,left_low,no


In [5]:
db.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 286 entries, 0 to 285
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   recurrence-events  286 non-null    bool  
 1   age                286 non-null    object
 2   menopause          286 non-null    object
 3   tumor-size         286 non-null    object
 4   inv-nodes          286 non-null    object
 5   node-caps          286 non-null    object
 6   deg-malig          286 non-null    int64 
 7   breast             286 non-null    object
 8   breast-quad        286 non-null    object
 9   irradiat           286 non-null    object
dtypes: bool(1), int64(1), object(8)
memory usage: 20.5+ KB


In [6]:
x_mat = db.iloc[:, 1:].astype('category')
y_vec = db.iloc[:, 0]

In [7]:
x_mat.apply(pd.unique)

age            ['30-39', '40-49', '60-69', '50-59', '70-79', ...
menopause      ['premeno', 'ge40', 'lt40']
Categories (3, obj...
tumor-size     ['30-34', '20-24', '15-19', '0-4', '25-29', .....
inv-nodes      ['0-2', '6-8', '9-11', '3-5', '15-17', '12-14'...
node-caps      ['no', 'yes', '?']
Categories (3, object): ['?...
deg-malig             [3, 2, 1]
Categories (3, int64): [1, 2, 3]
breast         ['left', 'right']
Categories (2, object): ['le...
breast-quad    ['left_low', 'right_up', 'left_up', 'right_low...
irradiat       ['no', 'yes']
Categories (2, object): ['no', '...
dtype: object

In [8]:
from sklearn.naive_bayes import CategoricalNB, ComplementNB, GaussianNB, MultinomialNB 

In [9]:
dx = OrdinalEncoder().fit_transform(x_mat)

In [10]:
dx

array([[1., 2., 5., ..., 0., 2., 0.],
       [2., 2., 3., ..., 1., 5., 0.],
       [2., 2., 3., ..., 0., 2., 0.],
       ...,
       [4., 0., 3., ..., 1., 3., 0.],
       [2., 0., 5., ..., 0., 2., 0.],
       [3., 0., 5., ..., 0., 2., 0.]])

<div dir="rtl">
    <font size=4>
در ابتدا روی کل دیتاست چهار مدل قابل اعمال با تنظیمات پیش فرض اجرا شده است. البته در اینجا داده ها به صورت دسته بندی هستند و در عمل استفاده از مدل گوشی که روی داده های پیوسته کاربردی است نیاز نبوده و تنها برای مقایسه عملکردی تست شده است.
    </font>
</div>

In [11]:
NB_models = {}
scores = [('model','score','f1-score')]
for model in [CategoricalNB, ComplementNB, GaussianNB, MultinomialNB]:
    name = str(model).split('.')[-1]
    NB_models[name] = model().fit(dx, y_vec)
    scores.append((name,
                   NB_models[name].score(dx, y_vec).round(2),
                   f1_score(y_true=y_vec, y_pred=NB_models[name].predict(dx)).round(2)
                  ))
scores

[('model', 'score', 'f1-score'),
 ("CategoricalNB'>", 0.76, 0.55),
 ("ComplementNB'>", 0.72, 0.5),
 ("GaussianNB'>", 0.73, 0.53),
 ("MultinomialNB'>", 0.72, 0.47)]

<div dir="rtl">
    <font size=4>
        تقریبن عملکرد هر چهار مدل یکسان است و مدل کاتگوریکال با توجه به بررسی احتمال داخل طبقه ی هر فیچر عملکردش به نظر از سه تای دیگه بیشتر شده اما نکته مهم این است که بخاطر درصد پایین این مدل ها روی این مسئله خروجی کم برازش با بایاس بالا دارند و به نظر من در این موضوع این مدل مناسب نیست.
        اما در ادامه به اثر تک تک فیچرها روی مدل پرداخته شده و مدل هایی با استفاده از فیچرهای مختلف ساخته شده است.
    </font>
</div>

In [12]:
from itertools import combinations

In [13]:
modes = {}
scores = {}
for i in range(2, 10):
    for j in combinations(range(9),i):
        modes[str(j)] = MultinomialNB().fit(dx[:, j], y_vec)
        scores[str(j)] = ( modes[str(j)].score(dx[:, j], y_vec).round(3),
                          f1_score(y_vec, modes[str(j)].predict(dx[:, j])).round(3),
                         )
scores

{'(0, 1)': (0.703, 0.0),
 '(0, 2)': (0.703, 0.0),
 '(0, 3)': (0.717, 0.456),
 '(0, 4)': (0.703, 0.0),
 '(0, 5)': (0.703, 0.0),
 '(0, 6)': (0.703, 0.0),
 '(0, 7)': (0.703, 0.0),
 '(0, 8)': (0.703, 0.0),
 '(1, 2)': (0.703, 0.0),
 '(1, 3)': (0.72, 0.394),
 '(1, 4)': (0.703, 0.0),
 '(1, 5)': (0.703, 0.0),
 '(1, 6)': (0.703, 0.0),
 '(1, 7)': (0.703, 0.0),
 '(1, 8)': (0.703, 0.0),
 '(2, 3)': (0.72, 0.467),
 '(2, 4)': (0.703, 0.0),
 '(2, 5)': (0.703, 0.0),
 '(2, 6)': (0.703, 0.0),
 '(2, 7)': (0.703, 0.0),
 '(2, 8)': (0.703, 0.0),
 '(3, 4)': (0.717, 0.372),
 '(3, 5)': (0.682, 0.133),
 '(3, 6)': (0.748, 0.41),
 '(3, 7)': (0.717, 0.456),
 '(3, 8)': (0.703, 0.0),
 '(4, 5)': (0.703, 0.0),
 '(4, 6)': (0.703, 0.0),
 '(4, 7)': (0.703, 0.0),
 '(4, 8)': (0.703, 0.0),
 '(5, 6)': (0.703, 0.0),
 '(5, 7)': (0.703, 0.0),
 '(5, 8)': (0.703, 0.0),
 '(6, 7)': (0.703, 0.0),
 '(6, 8)': (0.703, 0.0),
 '(7, 8)': (0.703, 0.0),
 '(0, 1, 2)': (0.703, 0.0),
 '(0, 1, 3)': (0.713, 0.453),
 '(0, 1, 4)': (0.703, 0.0),
 '(

<div dir="rtl">
    <font size=4>
    تعداد کل نمونه ها286 عدد است که تنها 85 تای آنها مثبت است. در نتیجه با تخمین ثابت منفی بودن (بردار ثابت صفر) صحت مدل 70%   است و درنتیجه در اینجا معیار خوبی نیست. همانطور هم که می بینید در داده های بالا بسیاری دارای چنین صحتی هستن اما معیار f1 آنها صفر بوده و نشان می دهد که در واقع هیچ مورد سرطانی را تشخیص ندادن
        بهترین صحت توسط مدل با سه فیچر بدست آمده اما بر اساس معیار دقت با 6 فیچر از 9 تا بدست اومده که تنها 0.48 است.
    </font>
</div>

In [14]:
sorted_scores = sorted(scores, key=scores.get)

In [15]:
sorted_scores[-1]

'(3, 6, 8)'

In [16]:
scores[sorted_scores[-1]]

(0.748, 0.41)

In [17]:
sorted_f1scores = sorted(scores, key=lambda x: scores[x][1])

In [18]:
sorted_f1scores[-1]

'(0, 1, 3, 5, 6, 8)'

In [19]:
scores[sorted_f1scores[-1]]

(0.72, 0.481)

<div dir="rtl">
    <font size=4>
    اینجا کل بررسی هم هم روی داده های تعلیم بوده و یعنی حتی در این مسئله مدل نایو بیز نتونسته روی داده های تعلیم هم خوب عمل کنه. در ادامه استفاده از مدل جنگل تصادفی با پارامترهای پیش فرض استفاده شده است.
    </font>
</div>

In [20]:
from sklearn.ensemble import RandomForestClassifier

In [21]:
forest = RandomForestClassifier().fit(dx, y_vec)

In [22]:
f'{forest.score(dx, y_vec).round(3)}  -  {f1_score(y_vec, forest.predict(dx)).round(3)}'

'0.979  -  0.964'

<div dir="rtl">
    <font size=4>
    کاملن مدل تونسته روی داده های تعلیم خودشو مچ کنه که این ویژگی عمومی این مدل هست و برای همین همواره ریسک بیش بردازش در آن وجود داره. پس اینجا داده هارو به حفظ توزیع به دو قیست تعلیم و تست تقسیم می کنیم و اثر مدل ها را روی داده های تست بررسی می کنیم.
    </font>
</div>

In [23]:
from sklearn.model_selection import train_test_split

In [41]:
x_train, x_test, y_train, y_test = train_test_split(dx, y_vec, stratify=y_vec, random_state=1224)

In [42]:
forest = RandomForestClassifier().fit(x_train, y_train)

In [43]:
print(f'\tscore\tf1-score\ntrain:\t{forest.score(x_train, y_train).round(3)}  -  {f1_score(y_train, forest.predict(x_train)).round(3)} \ntest:\t{forest.score(x_test, y_test).round(3)}  -  {f1_score(y_test, forest.predict(x_test)).round(3)}')

	score	f1-score
train:	0.977  -  0.961 
test:	0.694  -  0.45


<div dir="rtl">
    <font size=4>
    اینجا به راحتی نشون میده که با تنظیمات پایه مدل جنگل دارای اوور فیت بالایی هست. در ادامه هم اثر تقسیم دیتاست را روی نایو بیز بررسی کردیم
    </font>
</div>

In [44]:
NB_models = {}
scores = [('model','score','f1-score')]
for model in [CategoricalNB, ComplementNB, GaussianNB, MultinomialNB]:
    name = str(model).split('.')[-1]
    NB_models[name] = model().fit(x_train, y_train)
    scores.append((name,
                   NB_models[name].score(x_test, y_test).round(2),
                   f1_score(y_true=y_test, y_pred=NB_models[name].predict(x_test)).round(2)
                  ))
scores

[('model', 'score', 'f1-score'),
 ("CategoricalNB'>", 0.74, 0.49),
 ("ComplementNB'>", 0.76, 0.54),
 ("GaussianNB'>", 0.76, 0.59),
 ("MultinomialNB'>", 0.75, 0.5)]

In [45]:
all_modes = {}
all_scores = {}
for i in range(2, 10):
    for j in combinations(range(9),i):
        modes = {}
        scores = {}
        for model in [CategoricalNB, ComplementNB, GaussianNB, MultinomialNB]:
            name = str(model).split('.')[-1]
            modes[name] = model().fit(x_train[:, j], y_train)
            scores[name] = ( modes[name].score(x_test[:, j], y_test).round(3),
                              f1_score(y_test, modes[name].predict(x_test[:, j]), average=None, zero_division=1).round(3),
                             )
        all_modes[str(j)] = modes
        all_scores[str(j)] = scores

In [46]:
for xset, scores in all_scores.items():
    print(xset, end=':\n\t')
    print(*scores.items(),sep='\n\t')

(0, 1):
	("CategoricalNB'>", (0.708, array([0.829, 0.   ])))
	("ComplementNB'>", (0.472, array([0.525, 0.406])))
	("GaussianNB'>", (0.708, array([0.829, 0.   ])))
	("MultinomialNB'>", (0.708, array([0.829, 0.   ])))
(0, 2):
	("CategoricalNB'>", (0.681, array([0.803, 0.148])))
	("ComplementNB'>", (0.556, array([0.619, 0.467])))
	("GaussianNB'>", (0.708, array([0.829, 0.   ])))
	("MultinomialNB'>", (0.708, array([0.829, 0.   ])))
(0, 3):
	("CategoricalNB'>", (0.708, array([0.817, 0.276])))
	("ComplementNB'>", (0.75, array([0.833, 0.5  ])))
	("GaussianNB'>", (0.736, array([0.826, 0.457])))
	("MultinomialNB'>", (0.736, array([0.826, 0.457])))
(0, 4):
	("CategoricalNB'>", (0.694, array([0.804, 0.312])))
	("ComplementNB'>", (0.458, array([0.506, 0.4  ])))
	("GaussianNB'>", (0.681, array([0.793, 0.303])))
	("MultinomialNB'>", (0.708, array([0.829, 0.   ])))
(0, 5):
	("CategoricalNB'>", (0.736, array([0.822, 0.486])))
	("ComplementNB'>", (0.556, array([0.59 , 0.515])))
	("GaussianNB'>", (0.75,

	("CategoricalNB'>", (0.708, array([0.821, 0.222])))
	("ComplementNB'>", (0.528, array([0.585, 0.452])))
	("GaussianNB'>", (0.681, array([0.793, 0.303])))
	("MultinomialNB'>", (0.708, array([0.829, 0.   ])))
(1, 4, 6, 8):
	("CategoricalNB'>", (0.694, array([0.81 , 0.214])))
	("ComplementNB'>", (0.583, array([0.688, 0.375])))
	("GaussianNB'>", (0.681, array([0.789, 0.343])))
	("MultinomialNB'>", (0.708, array([0.829, 0.   ])))
(1, 4, 7, 8):
	("CategoricalNB'>", (0.694, array([0.81 , 0.214])))
	("ComplementNB'>", (0.639, array([0.755, 0.316])))
	("GaussianNB'>", (0.667, array([0.782, 0.294])))
	("MultinomialNB'>", (0.708, array([0.829, 0.   ])))
(1, 5, 6, 7):
	("CategoricalNB'>", (0.792, array([0.867, 0.516])))
	("ComplementNB'>", (0.611, array([0.667, 0.533])))
	("GaussianNB'>", (0.75, array([0.845, 0.357])))
	("MultinomialNB'>", (0.708, array([0.829, 0.   ])))
(1, 5, 6, 8):
	("CategoricalNB'>", (0.75, array([0.836, 0.471])))
	("ComplementNB'>", (0.653, array([0.737, 0.49 ])))
	("Gaussi

	("CategoricalNB'>", (0.764, array([0.838, 0.564])))
	("ComplementNB'>", (0.625, array([0.682, 0.542])))
	("GaussianNB'>", (0.792, array([0.862, 0.571])))
	("MultinomialNB'>", (0.708, array([0.829, 0.   ])))
(1, 2, 5, 6, 8):
	("CategoricalNB'>", (0.778, array([0.846, 0.6  ])))
	("ComplementNB'>", (0.639, array([0.723, 0.48 ])))
	("GaussianNB'>", (0.681, array([0.781, 0.41 ])))
	("MultinomialNB'>", (0.708, array([0.829, 0.   ])))
(1, 2, 5, 7, 8):
	("CategoricalNB'>", (0.722, array([0.818, 0.412])))
	("ComplementNB'>", (0.569, array([0.659, 0.415])))
	("GaussianNB'>", (0.681, array([0.785, 0.378])))
	("MultinomialNB'>", (0.708, array([0.829, 0.   ])))
(1, 2, 6, 7, 8):
	("CategoricalNB'>", (0.681, array([0.8  , 0.207])))
	("ComplementNB'>", (0.569, array([0.674, 0.367])))
	("GaussianNB'>", (0.667, array([0.782, 0.294])))
	("MultinomialNB'>", (0.708, array([0.829, 0.   ])))
(1, 3, 4, 5, 6):
	("CategoricalNB'>", (0.764, array([0.847, 0.485])))
	("ComplementNB'>", (0.75, array([0.833, 0.5  ]

In [47]:
def deep (m):
    title = max(m, key=lambda x:m[x][1][1])
    return m[title][1][1]

In [48]:
fiq = max(all_scores, key=lambda m: deep(all_scores[m]))
fiq

'(0, 2, 5)'

In [49]:
all_scores[fiq]

{"CategoricalNB'>": (0.778, array([0.843, 0.619])),
 "ComplementNB'>": (0.597, array([0.633, 0.554])),
 "GaussianNB'>": (0.75, array([0.827, 0.55 ])),
 "MultinomialNB'>": (0.708, array([0.829, 0.   ]))}

<div dir="rtl">
    <font size=4>
    اتفاق جالبی که اینجا افتاده با کاهش اطلاعات مدل در مرحله تعلیم، با کمال تعحب نتیجه روی داده های تست بهتر بوده ولی به بالاترین مقدار در حالت های مختلف به 0.57 رسیده که البته همچنان کم است.
        یک مسئله که شاید اینجا مطرح هست خوب نبودن اطلاعات است و بایستی اطلاعات ورودی پالایش شود که با توجه به موضوع پزشکی مسئله ذهنیتی در این راستا ندارم.
    </font>
</div>

<div dir="rtl">
    <font color=red size=4>
        یک مسئله مهم که در تکرار این فایل به آن برخوردم نحوه تقسیم داده های تست و ترین بود که متوجه شدم با توجه به کم بودن داده ها، ترتیب تقسیم اون ها تاثیر شدیدی در نتیجه مدل ها گذاشته و درنتیجه به نظر میرسه که روش تقسیم تست و ترین خیلی برای بررسی مدل خوب نباشه و شاید روشی های مبتنی بر کل دیتاست مثل روش kfold بهتر بتونند این مسئله را نشون بدن.
    </font>
</div>

In [50]:
print(f'recurrence-events in:\n\t train:{y_train.sum()}, test:{y_test.sum()}\n',
      f'sample size \n\t train:{y_train.shape[0]}, test:{y_test.shape[0]}')

recurrence-events in:
	 train:64, test:21
 sample size 
	 train:214, test:72


In [51]:
for i in range(2, 10):
    for j in combinations(range(9),i):
        modes = {}
        scores = {}
        for name in [1e-10,0.1,2]:
            modes[name] = ComplementNB(alpha=name).fit(x_train[:, j], y_train)
            scores[name] = ( modes[name].score(x_test[:, j], y_test).round(3),
                              f1_score(y_test, modes[name].predict(x_test[:, j]), average=None, zero_division=1).round(3),
                             )
        all_modes[str(j)] = modes
        all_scores[str(j)] = scores

In [52]:
for xset, scores in all_scores.items():
    print(xset, end=':\n\t')
    print(*scores.items(),sep='\n\t')

(0, 1):
	(1e-10, (0.472, array([0.525, 0.406])))
	(0.1, (0.472, array([0.525, 0.406])))
	(2, (0.472, array([0.525, 0.406])))
(0, 2):
	(1e-10, (0.556, array([0.619, 0.467])))
	(0.1, (0.556, array([0.619, 0.467])))
	(2, (0.556, array([0.619, 0.467])))
(0, 3):
	(1e-10, (0.75, array([0.833, 0.5  ])))
	(0.1, (0.75, array([0.833, 0.5  ])))
	(2, (0.75, array([0.833, 0.5  ])))
(0, 4):
	(1e-10, (0.458, array([0.506, 0.4  ])))
	(0.1, (0.458, array([0.506, 0.4  ])))
	(2, (0.458, array([0.506, 0.4  ])))
(0, 5):
	(1e-10, (0.556, array([0.59 , 0.515])))
	(0.1, (0.556, array([0.59 , 0.515])))
	(2, (0.556, array([0.59 , 0.515])))
(0, 6):
	(1e-10, (0.5, array([0.571, 0.4  ])))
	(0.1, (0.5, array([0.571, 0.4  ])))
	(2, (0.5, array([0.571, 0.4  ])))
(0, 7):
	(1e-10, (0.569, array([0.652, 0.436])))
	(0.1, (0.569, array([0.652, 0.436])))
	(2, (0.569, array([0.652, 0.436])))
(0, 8):
	(1e-10, (0.653, array([0.766, 0.324])))
	(0.1, (0.653, array([0.766, 0.324])))
	(2, (0.653, array([0.766, 0.324])))
(1, 2):
	

	(2, (0.75, array([0.833, 0.5  ])))
(0, 1, 3, 5, 6):
	(1e-10, (0.75, array([0.833, 0.5  ])))
	(0.1, (0.75, array([0.833, 0.5  ])))
	(2, (0.75, array([0.833, 0.5  ])))
(0, 1, 3, 5, 7):
	(1e-10, (0.75, array([0.833, 0.5  ])))
	(0.1, (0.75, array([0.833, 0.5  ])))
	(2, (0.75, array([0.833, 0.5  ])))
(0, 1, 3, 5, 8):
	(1e-10, (0.764, array([0.841, 0.541])))
	(0.1, (0.764, array([0.841, 0.541])))
	(2, (0.764, array([0.841, 0.541])))
(0, 1, 3, 6, 7):
	(1e-10, (0.75, array([0.833, 0.5  ])))
	(0.1, (0.75, array([0.833, 0.5  ])))
	(2, (0.75, array([0.833, 0.5  ])))
(0, 1, 3, 6, 8):
	(1e-10, (0.764, array([0.841, 0.541])))
	(0.1, (0.764, array([0.841, 0.541])))
	(2, (0.764, array([0.841, 0.541])))
(0, 1, 3, 7, 8):
	(1e-10, (0.75, array([0.833, 0.5  ])))
	(0.1, (0.75, array([0.833, 0.5  ])))
	(2, (0.75, array([0.833, 0.5  ])))
(0, 1, 4, 5, 6):
	(1e-10, (0.583, array([0.625, 0.531])))
	(0.1, (0.583, array([0.625, 0.531])))
	(2, (0.583, array([0.625, 0.531])))
(0, 1, 4, 5, 7):
	(1e-10, (0.542, arra

	(2, (0.75, array([0.833, 0.5  ])))
(0, 3, 5, 6, 7, 8):
	(1e-10, (0.764, array([0.841, 0.541])))
	(0.1, (0.764, array([0.841, 0.541])))
	(2, (0.764, array([0.841, 0.541])))
(0, 4, 5, 6, 7, 8):
	(1e-10, (0.639, array([0.711, 0.519])))
	(0.1, (0.639, array([0.711, 0.519])))
	(2, (0.625, array([0.703, 0.491])))
(1, 2, 3, 4, 5, 6):
	(1e-10, (0.75, array([0.833, 0.5  ])))
	(0.1, (0.75, array([0.833, 0.5  ])))
	(2, (0.75, array([0.833, 0.5  ])))
(1, 2, 3, 4, 5, 7):
	(1e-10, (0.75, array([0.833, 0.5  ])))
	(0.1, (0.75, array([0.833, 0.5  ])))
	(2, (0.75, array([0.833, 0.5  ])))
(1, 2, 3, 4, 5, 8):
	(1e-10, (0.736, array([0.822, 0.486])))
	(0.1, (0.736, array([0.822, 0.486])))
	(2, (0.736, array([0.822, 0.486])))
(1, 2, 3, 4, 6, 7):
	(1e-10, (0.75, array([0.833, 0.5  ])))
	(0.1, (0.75, array([0.833, 0.5  ])))
	(2, (0.75, array([0.833, 0.5  ])))
(1, 2, 3, 4, 6, 8):
	(1e-10, (0.75, array([0.833, 0.5  ])))
	(0.1, (0.75, array([0.833, 0.5  ])))
	(2, (0.75, array([0.833, 0.5  ])))
(1, 2, 3, 4, 7, 8

<div dir="rtl">
    <font size=4>
    به عملکرد مدل LogisticRegression هم در ادامه یک نیم نگاهی کنیم.
    </font>
</div>

In [53]:
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV

In [54]:
logregclf = LogisticRegression().fit(x_train, y_train)

In [55]:
f1_score(y_test, logregclf.predict(x_test))

0.35714285714285715

In [56]:
logregclfcv = LogisticRegressionCV(cv=10, Cs=[0.1, 1, 10, 20, 30, 40, 80, 100], solver='liblinear', scoring='f1', penalty='l1').fit(x_train, y_train)

In [57]:
f1_score(y_test, logregclfcv.predict(x_test))

0.35714285714285715

In [70]:
logregclfcvall = LogisticRegressionCV(cv=10, Cs=1003, solver='liblinear', scoring='f1', penalty='l1').fit(dx, y_vec)

In [71]:
f1_score(y_vec, logregclfcvall.predict(dx))

0.48062015503875966

<div dir="rtl">
    <font size=4>
        بررسی روی کل داده ها به روش لاجیستیک رگرشن هم نشون داد که این روش هم نتونسته بازهی خوبی داشته باشه و شدیدن دارای underfitting میشه که بنظر یکی از دلایل کم بودن نمونه ها نسبت به گستره انواع فیچرهاست 
    </font>
</div>

In [72]:
from sklearn import svm

In [96]:
C = 10.0  # SVM regularization parameter
models = (svm.SVC(kernel='linear', C=C),
          svm.SVC(kernel='rbf', gamma=0.7, C=C),
          svm.SVC(kernel='poly', degree=3, gamma='auto', C=C))
models = (clf.fit(x_train, y_train) for clf in models)

In [97]:
models_f1 = [f1_score(y_train, clf.predict(x_train)) for clf in models]
models_f1

[0.4770642201834862, 0.9612403100775193, 0.8521739130434782]

<div dir="rtl">
    <font size=4>
    اینجا میبینیم که مدل SVM بدون بررسی دقیق ضرائب و با مقادیر پایه با کرنل غیر خطی عملکر قوی را نشون میده
    </font>
</div>

In [109]:
C_list = [ 0.001, 0.01, 0.1, 1, 10]
Gamma_list = [0.01, 0.1, 0.5, 1, 2, 10]
models_rbf = (svm.SVC(kernel='rbf', gamma=g, C=c) for c in C_list for g in Gamma_list)
models_rbf = [clf.fit(x_train, y_train) for clf in models_rbf]
models_rbf_scores = [f1_score(y_test, clf.predict(x_test)) for clf in models_rbf]
max(models_rbf_scores)

0.5531914893617021

In [119]:
models_ply = (svm.SVC(kernel='poly', degree=degree, gamma='auto', C=c) for c in C_list for degree in [2, 3, 4, 5])
models_ply = [clf.fit(x_train, y_train) for clf in models_ply]
models_ply_scores = [f1_score(y_test, clf.predict(x_test)) for clf in models_ply]
print('max f1 score:', max(models_ply_scores))
sets = {scr:mdl for scr, mdl in zip(models_ply_scores, models_ply)}
print(sets[max(sets)])

0.6315789473684211
SVC(C=0.1, degree=4, gamma='auto', kernel='poly')


In [123]:
C_list = [1e-5, 1e-4, 1e-3, 0.01]
Gamma_list = [0.01, 0.1, 0.5, 1, 2, 10]
models_ply = (svm.SVC(kernel='poly', degree=4, gamma=g, C=c) for c in C_list for g in Gamma_list)
models_ply = [clf.fit(x_train, y_train) for clf in models_ply]
models_ply_scores = [f1_score(y_test, clf.predict(x_test)) for clf in models_ply]
print('max f1 score:', max(models_ply_scores))
sets = {scr:mdl for scr, mdl in zip(models_ply_scores, models_ply)}
print(sets[max(sets)])

max f1 score: 0.6222222222222222
SVC(C=0.001, degree=4, gamma=0.5, kernel='poly')


In [124]:
final = svm.SVC(kernel='poly', degree=4, gamma=0.5, C=0.001)
final.fit(x_train, y_train)
print('f1_train:', f1_score(y_train, final.predict(x_train)))
print('f1_test:', f1_score(y_test, final.predict(x_test)))
final.fit(dx, y_vec)
print('f1_ full data train:', f1_score(y_vec, final.predict(dx)))

f1_train: 0.8288288288288289
f1_test: 0.6222222222222222
f1_ full data train: 0.810810810810811
